## Libraries

In [1]:
import sys
sys.path.append('./venv/lib/python3.6/site-packages')

In [2]:
import os
import time
import random

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

In [3]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms as vtransforms
from torchvision import utils as vutils
from torchvision import datasets

In [4]:
from gans.utils.data import CartoonSet
from gans.utils.layers import *
from gans.utils.functions import *
from gans.models import *
from gans.trainer import *

## Random seed

In [5]:
# Set random seem for reproducibility
# manualSeed = 999
manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  488


## Inputs

In [6]:
# Dataset
data_root = './datasets'

# data_name = 'FashionMNIST'
# data_name = 'MNIST'

# batch_size = 200
# sample_size = 50
# nrow = 5

data_name = 'CartoonSet'

batch_size = 128
sample_size = 32
nrow = 4

num_epochs = 50

num_workers = 8


# Output dir
output_root = './output/vanillaGAN'
# overwrite real sample output
overwrite_real = False

noize_dim = 100

# Optimization parameters
lr = 0.001
beta1 = 0.5

## Device

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

print('using device: ', device)

using device:  cuda:1


## Dataset

In [8]:
img_size = 28
in_channels = 1

if data_name == 'CartoonSet':
    img_size = 75
    dataset = CartoonSet(root=os.path.join(data_root, data_name), 
                         transform=vtransforms.Compose([
                             vtransforms.CenterCrop(size=400),
                             vtransforms.Resize(size=img_size), 
                             vtransforms.ToTensor(),
                             vtransforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
                         ]))
    
elif data_name == 'MNIST':
    dataset = datasets.MNIST(root=os.path.join(data_root, data_name), 
                             download=True,
                             transform=vtransforms.Compose([
                                 vtransforms.ToTensor(),
                                 vtransforms.Normalize((0.5,), (0.5,)),
                         ]))
elif data_name == 'FashionMNIST':
    dataset = datasets.FashionMNIST(root=os.path.join(data_root, data_name), 
                                    download=True,
                                    transform=vtransforms.Compose([
                                        vtransforms.ToTensor(),
                                        vtransforms.Normalize((0.5,), (0.5,)),
                                    ]))
    
dtype = dataset[0][0].dtype
img_shape = dataset[0][0].shape

## Dataloader

In [9]:
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=batch_size,
                                         shuffle=True, 
                                         num_workers=num_workers)

In [10]:
if not os.path.exists(os.path.join(output_root, data_name)):
    os.makedirs(os.path.join(output_root, data_name))

real_sample_path = os.path.join(output_root, data_name, 'real_samples.png')


real_batch = next(iter(dataloader))

vutils.save_image(real_batch[0], real_sample_path, 
                  normalize=True, range=(0,1), nrow=nrow)
Image(real_sample_path)

## Model

In [11]:
C, H, W = img_shape
features = C * H * W

In [12]:
layersG = [
    *linearBlock(noize_dim, 128, normalize=False, activation='ReLU'),
    *linearBlock(128, 256, activation='ReLU'),
    *linearBlock(256, 512, activation='ReLU'),
    *linearBlock(512, 1024, activation='ReLU'),
    *linearBlock(1024, features, normalize=False, activation='Tanh'),
]

netG = Generator(layers=layersG, img_shape=img_shape)
print(netG)

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Linear(in_features=1024, out_features=16875, bias=True)
    (12): Tanh()
    (13): Unflatten2d(img_shape=(3, 75, 75))
  )
)


In [13]:
layersD = [
    *linearBlock(features, 1024, normalize=False, activation='LeakyReLU'),
    *linearBlock(1024, 512, normalize=False, activation='LeakyReLU'),
    *linearBlock(512, 256, normalize=False, activation='LeakyReLU'),
    *linearBlock(256, 128, normalize=False, activation='LeakyReLU'),
    *linearBlock(128, 1, normalize=False, activation='Sigmoid'),
]

netD = Discriminator(layers=layersD)
print(netD)

Discriminator(
  (model): Sequential(
    (0): Flatten()
    (1): Linear(in_features=16875, out_features=1024, bias=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Linear(in_features=512, out_features=256, bias=True)
    (6): LeakyReLU(negative_slope=0.2, inplace)
    (7): Linear(in_features=256, out_features=128, bias=True)
    (8): LeakyReLU(negative_slope=0.2, inplace)
    (9): Linear(in_features=128, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


## Training

In [14]:
trainer = GANTrainer(generator=netG, discriminator=netD,
                     optimizer=optim.Adam, loss=nn.BCELoss(), 
                     device=device, multi_gpu=False, output_path=os.path.join(output_root, data_name),
                     batch_size=batch_size, num_epochs=num_epochs, sample_size=sample_size)

In [15]:
trainer.train(dataloader=dataloader, nrow=nrow)

/home/shu/.local/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[Epoch 0/50][Batch 0/79] Loss_D: 1.39 Loss_G: 0.92
[Epoch 0/50][Batch 15/79] Loss_D: 6.34 Loss_G: 0.58
[Epoch 0/50][Batch 30/79] Loss_D: 1.04 Loss_G: 0.68
[Epoch 0/50][Batch 45/79] Loss_D: 2.65 Loss_G: 0.50
[Epoch 0/50][Batch 60/79] Loss_D: 0.44 Loss_G: 1.56
[Epoch 0/50][Batch 75/79] Loss_D: 0.50 Loss_G: 1.45
Epoch 0 running time: 84.74


/home/shu/.local/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[Epoch 1/50][Batch 0/79] Loss_D: 0.62 Loss_G: 1.75
[Epoch 1/50][Batch 15/79] Loss_D: 0.54 Loss_G: 2.19
[Epoch 1/50][Batch 30/79] Loss_D: 0.48 Loss_G: 1.44
[Epoch 1/50][Batch 45/79] Loss_D: 0.73 Loss_G: 1.32
[Epoch 1/50][Batch 60/79] Loss_D: 1.03 Loss_G: 1.26
[Epoch 1/50][Batch 75/79] Loss_D: 0.98 Loss_G: 1.79
Epoch 1 running time: 5.79
[Epoch 2/50][Batch 0/79] Loss_D: 1.85 Loss_G: 3.09
[Epoch 2/50][Batch 15/79] Loss_D: 1.25 Loss_G: 1.27
[Epoch 2/50][Batch 30/79] Loss_D: 10.03 Loss_G: 0.29
[Epoch 2/50][Batch 45/79] Loss_D: 1.16 Loss_G: 0.50
[Epoch 2/50][Batch 60/79] Loss_D: 1.74 Loss_G: 0.86
[Epoch 2/50][Batch 75/79] Loss_D: 1.37 Loss_G: 0.54
Epoch 2 running time: 6.70
[Epoch 3/50][Batch 0/79] Loss_D: 1.26 Loss_G: 0.71
[Epoch 3/50][Batch 15/79] Loss_D: 1.11 Loss_G: 2.17
[Epoch 3/50][Batch 30/79] Loss_D: 1.43 Loss_G: 0.59
[Epoch 3/50][Batch 45/79] Loss_D: 1.15 Loss_G: 0.93
[Epoch 3/50][Batch 60/79] Loss_D: 1.18 Loss_G: 0.94
[Epoch 3/50][Batch 75/79] Loss_D: 1.34 Loss_G: 0.79
Epoch 3 runn

[Epoch 25/50][Batch 0/79] Loss_D: 1.39 Loss_G: 0.85
[Epoch 25/50][Batch 15/79] Loss_D: 1.96 Loss_G: 0.79
[Epoch 25/50][Batch 30/79] Loss_D: 1.38 Loss_G: 0.71
[Epoch 25/50][Batch 45/79] Loss_D: 1.37 Loss_G: 0.66
[Epoch 25/50][Batch 60/79] Loss_D: 1.44 Loss_G: 0.61
[Epoch 25/50][Batch 75/79] Loss_D: 1.38 Loss_G: 0.68
Epoch 25 running time: 7.13
[Epoch 26/50][Batch 0/79] Loss_D: 1.34 Loss_G: 0.70
[Epoch 26/50][Batch 15/79] Loss_D: 1.38 Loss_G: 0.68
[Epoch 26/50][Batch 30/79] Loss_D: 1.37 Loss_G: 0.71
[Epoch 26/50][Batch 45/79] Loss_D: 1.35 Loss_G: 0.91
[Epoch 26/50][Batch 60/79] Loss_D: 1.37 Loss_G: 0.50
[Epoch 26/50][Batch 75/79] Loss_D: 1.35 Loss_G: 0.62
Epoch 26 running time: 6.83
[Epoch 27/50][Batch 0/79] Loss_D: 1.37 Loss_G: 0.71
[Epoch 27/50][Batch 15/79] Loss_D: 1.31 Loss_G: 0.68
[Epoch 27/50][Batch 30/79] Loss_D: 1.38 Loss_G: 0.70
[Epoch 27/50][Batch 45/79] Loss_D: 1.35 Loss_G: 0.71
[Epoch 27/50][Batch 60/79] Loss_D: 1.34 Loss_G: 0.71
[Epoch 27/50][Batch 75/79] Loss_D: 1.29 Loss_G

[Epoch 48/50][Batch 15/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 48/50][Batch 30/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 48/50][Batch 45/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 48/50][Batch 60/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 48/50][Batch 75/79] Loss_D: 27.63 Loss_G: 27.63
Epoch 48 running time: 6.73
[Epoch 49/50][Batch 0/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 49/50][Batch 15/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 49/50][Batch 30/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 49/50][Batch 45/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 49/50][Batch 60/79] Loss_D: 27.63 Loss_G: 27.63
[Epoch 49/50][Batch 75/79] Loss_D: 27.63 Loss_G: 27.63
Epoch 49 running time: 7.00
Total running time: 433.67s


In [16]:
plt.figure(figsize=(8,3))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(trainer.g_losses,label="G")
plt.plot(trainer.d_losses,label="D")

plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig(os.path.join(output_root, data_name, 'g_d_loss.png'))
plt.show()

In [17]:
gif_generator(image_dir=os.path.join(output_root, data_name), 
              name_pattern=r'fake_samples_epoch_[\d]+|.png')

In [18]:
image_sample_dir = 'samples'

fake_name = os.path.join(data_name, 'vanillaGAN')

if not os.path.exists(os.path.join(image_sample_dir, fake_name)):
    os.makedirs(os.path.join(image_sample_dir, fake_name))

image_path = os.path.join(image_sample_dir, fake_name, 'img{:04}.png')


# switch to eval mode
netG.eval()

for i in range(2048):
    noise = torch.randn(1, noize_dim, 
                        dtype=dtype, device=device,
                        requires_grad=False)
    img = netG(noise)
    vutils.save_image(img, image_path.format(i), 
                      normalize=True, range=(0,1))